In [6]:
# Import necessary libraries
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Creates the data for the example and prints it."""
    data = {}
    # Technicians
    data['technicians'] = [
        {'id': 0, 'skill': 'Overhead', 'start_location': (0, 0)},
        {'id': 1, 'skill': 'Distribution', 'start_location': (10, 0)},
        {'id': 2, 'skill': 'Overhead', 'start_location': (5, 5)},
    ]
    data['num_technicians'] = len(data['technicians'])

    # Faults
    data['faults'] = [
        {'id': 0, 'skill_required': 'Overhead', 'location': (1, 1), 'time_window': (0, 480), 'duration': 60},
        {'id': 1, 'skill_required': 'Distribution', 'location': (2, -1), 'time_window': (0, 480), 'duration': 120},
        {'id': 2, 'skill_required': 'Overhead', 'location': (3, 3), 'time_window': (0, 480), 'duration': 90},
        {'id': 3, 'skill_required': 'Distribution', 'location': (6, 0), 'time_window': (0, 480), 'duration': 30},
        {'id': 4, 'skill_required': 'Overhead', 'location': (7, 8), 'time_window': (0, 480), 'duration': 45},
    ]
    data['num_faults'] = len(data['faults'])

    # Locations: Technicians' starting points + Faults' locations
    data['locations'] = [tech['start_location'] for tech in data['technicians']] + \
                        [fault['location'] for fault in data['faults']]

    data['num_locations'] = len(data['locations'])

    # Distance matrix (using Euclidean distance)
    data['distance_matrix'] = compute_euclidean_distance_matrix(data['locations'])

    # Time matrix (assuming time equals distance)
    data['time_matrix'] = data['distance_matrix']

    # Service times (duration)
    data['service_times'] = [0] * data['num_technicians'] + [fault['duration'] for fault in data['faults']]

    # Time windows
    # Technicians' working hours: (0, 480) minutes
    data['time_windows'] = [(0, 480)] * data['num_locations']
    # Update faults' time windows
    for i, fault in enumerate(data['faults']):
        index = data['num_technicians'] + i
        data['time_windows'][index] = fault['time_window']

    # Skills at each location
    data['skills'] = [tech['skill'] for tech in data['technicians']] + \
                     [fault['skill_required'] for fault in data['faults']]

    # Print the dataset for understanding
    print_problem_data(data)

    return data

def compute_euclidean_distance_matrix(locations):
    """Computes the Euclidean distance matrix."""
    import math
    distances = []
    for from_node in locations:
        row = []
        for to_node in locations:
            distance = math.hypot(from_node[0] - to_node[0], from_node[1] - to_node[1])
            row.append(int(distance))
        distances.append(row)
    return distances

def print_problem_data(data):
    """Prints the technicians and faults data."""
    print("\nTechnicians:")
    for tech in data['technicians']:
        print(f"  Technician {tech['id']}:")
        print(f"    Skill: {tech['skill']}")
        print(f"    Start Location: {tech['start_location']}")
        print(f"    Working Hours: {data['time_windows'][tech['id']]}")
        print()

    print("Faults:")
    for i, fault in enumerate(data['faults']):
        idx = data['num_technicians'] + i
        print(f"  Fault {fault['id']}:")
        print(f"    Required Skill: {fault['skill_required']}")
        print(f"    Location: {fault['location']}")
        print(f"    Time Window: {data['time_windows'][idx]}")
        print(f"    Expected Duration: {fault['duration']} minutes")
        print()

def main():
    """Solves the technician routing problem."""
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(
        data['num_locations'],
        data['num_technicians'],
        list(range(data['num_technicians'])),
        list(range(data['num_technicians']))
    )

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback (distance as time)
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes plus service time at from_node."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = data['time_matrix'][from_node][to_node]
        service_time = data['service_times'][from_node]
        return travel_time + service_time

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint
    routing.AddDimension(
        transit_callback_index,
        480,  # maximum waiting time
        480,  # maximum time per technician
        False,  # Don't force start cumul to zero
        'Time'
    )
    time_dimension = routing.GetDimensionOrDie('Time')

    # Add time window constraints for each location
    for location_idx in range(data['num_locations']):
        index = manager.NodeToIndex(location_idx)
        time_window = data['time_windows'][location_idx]
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Skill constraints
    # For each fault, specify which technicians can serve it
    for fault_idx in range(data['num_faults']):
        node_index = data['num_technicians'] + fault_idx
        required_skill = data['skills'][node_index]
        allowed_technicians = [t for t in range(data['num_technicians']) if data['skills'][t] == required_skill]
        if allowed_technicians:
            routing.SetAllowedVehiclesForIndex(allowed_technicians, manager.NodeToIndex(node_index))
        else:
            # Fault cannot be served by any technician, add disjunction with high penalty
            routing.AddDisjunction([manager.NodeToIndex(node_index)], 100000)

    # Allow dropping nodes with penalty
    penalty = 10000
    for node in range(data['num_technicians'], data['num_locations']):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = 10
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found!')

def print_solution(data, manager, routing, solution):
    """Prints the solution."""
    print('\nSolution:')
    total_time = 0
    for technician_id in range(data['num_technicians']):
        index = routing.Start(technician_id)
        plan_output = 'Technician {} (Skill: {}):\n'.format(
            technician_id, data['skills'][technician_id])
        route_time = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            time_var = routing.GetDimensionOrDie('Time').CumulVar(index)
            arrival_time = solution.Min(time_var)
            if node_index < data['num_technicians']:
                location_type = 'Start'
            else:
                location_type = 'Fault'
            plan_output += '  {0} at Location {1} (Time: {2})\n'.format(
                location_type, node_index, arrival_time)
            index = solution.Value(routing.NextVar(index))
            route_time = arrival_time
        time_var = routing.GetDimensionOrDie('Time').CumulVar(index)
        arrival_time = solution.Min(time_var)
        plan_output += '  End at Location {} (Time: {})\n'.format(
            manager.IndexToNode(index), arrival_time)
        plan_output += 'Total time: {} minutes\n'.format(route_time)
        print(plan_output)
        total_time += route_time
    print('Total time of all routes: {} minutes'.format(total_time))

if __name__ == '__main__':
    main()



Technicians:
  Technician 0:
    Skill: Overhead
    Start Location: (0, 0)
    Working Hours: (0, 480)

  Technician 1:
    Skill: Distribution
    Start Location: (10, 0)
    Working Hours: (0, 480)

  Technician 2:
    Skill: Overhead
    Start Location: (5, 5)
    Working Hours: (0, 480)

Faults:
  Fault 0:
    Required Skill: Overhead
    Location: (1, 1)
    Time Window: (0, 480)
    Expected Duration: 60 minutes

  Fault 1:
    Required Skill: Distribution
    Location: (2, -1)
    Time Window: (0, 480)
    Expected Duration: 120 minutes

  Fault 2:
    Required Skill: Overhead
    Location: (3, 3)
    Time Window: (0, 480)
    Expected Duration: 90 minutes

  Fault 3:
    Required Skill: Distribution
    Location: (6, 0)
    Time Window: (0, 480)
    Expected Duration: 30 minutes

  Fault 4:
    Required Skill: Overhead
    Location: (7, 8)
    Time Window: (0, 480)
    Expected Duration: 45 minutes


Solution:
Technician 0 (Skill: Overhead):
  Start at Location 0 (Time: 0)
  